# Exercises 03: Build Your Own DuckDBEngine

## 🎯 Goal

Implement a fully functional **DuckDBEngine** that:
- Inherits from Odibi's Engine ABC
- Uses DuckDB's analytical capabilities
- Supports CSV, Parquet, and JSON formats
- Leverages DuckDB's zero-copy Pandas integration
- Demonstrates a third execution model (analytical, in-process)

## 📋 Background

**Why DuckDB?**
- Fast analytical queries (10-100x faster than Pandas on aggregations)
- SQL-first design (optimized for SELECT, GROUP BY, JOIN)
- Zero-copy integration with Pandas/Arrow
- Single-process (no cluster needed)
- Perfect for 1GB - 100GB datasets

**Use Cases:**
- Analytical pipelines (BI, reporting)
- SQL-heavy transformations
- When Spark is overkill but Pandas is too slow

## 🏗️ Exercise 1: Set Up the Class Structure

Create the basic DuckDBEngine class that inherits from Engine ABC.

**Tasks:**
1. Import the Engine ABC from Odibi
2. Create DuckDBEngine class
3. Implement `__init__()` to create a DuckDB connection
4. Add docstring explaining the engine's purpose

In [ ]:
# Your code here
import sys
from pathlib import Path

# Add Odibi to path
odibi_path = Path(r"c:/Users/hodibi/OneDrive - Ingredion/Desktop/Repos/Odibi")
sys.path.insert(0, str(odibi_path))

from typing import Any, Dict, List, Optional
import duckdb
import pandas as pd

# TODO: Import Engine ABC

# TODO: Create DuckDBEngine class
class DuckDBEngine:
    """DuckDB-based execution engine.
    
    Optimized for analytical workloads on medium-sized datasets (1GB - 100GB).
    Uses DuckDB's columnar storage and vectorized execution for fast aggregations,
    joins, and window functions.
    """
    
    def __init__(self, db_path: str = ":memory:"):
        """Initialize DuckDB engine.
        
        Args:
            db_path: Path to DuckDB file, or ":memory:" for in-memory
        """
        # TODO: Create DuckDB connection
        pass

## 🏗️ Exercise 2: Implement read()

Implement the `read()` method using DuckDB's optimized readers.

**DuckDB Reading Patterns:**
```python
# CSV with auto-detection
df = conn.execute("SELECT * FROM read_csv_auto('file.csv')").df()

# Parquet
df = conn.execute("SELECT * FROM read_parquet('file.parquet')").df()

# JSON
df = conn.execute("SELECT * FROM read_json_auto('file.json')").df()
```

**Tasks:**
1. Get full path from connection (use `connection.get_path()`)
2. Implement format dispatch (csv, parquet, json)
3. Use DuckDB's read functions
4. Return Pandas DataFrame (`.df()`)
5. Raise helpful error for unsupported formats

In [ ]:
# Add this method to DuckDBEngine

def read(
    self,
    connection: Any,
    format: str,
    table: Optional[str] = None,
    path: Optional[str] = None,
    options: Optional[Dict[str, Any]] = None,
) -> pd.DataFrame:
    """Read data using DuckDB.
    
    Args:
        connection: Connection object (with get_path method)
        format: Data format (csv, parquet, json)
        table: Table name
        path: File path
        options: Format-specific options (ignored for now)
        
    Returns:
        Pandas DataFrame
    """
    # TODO: Implement
    pass

## 🏗️ Exercise 3: Implement write()

Implement the `write()` method using DuckDB's COPY command.

**DuckDB Writing Patterns:**
```python
# Register DataFrame
conn.register('temp_df', df)

# Write CSV
conn.execute(f"COPY temp_df TO 'file.csv' (HEADER, DELIMITER ',')")

# Write Parquet
conn.execute(f"COPY temp_df TO 'file.parquet' (FORMAT PARQUET)")
```

**Tasks:**
1. Get full path from connection
2. Register DataFrame with DuckDB (temporary name)
3. Use COPY command for each format
4. Handle overwrite mode (delete existing file first)
5. Handle append mode (use COPY ... (HEADER false) for CSV)

In [ ]:
# Add this method to DuckDBEngine

def write(
    self,
    df: pd.DataFrame,
    connection: Any,
    format: str,
    table: Optional[str] = None,
    path: Optional[str] = None,
    mode: str = "overwrite",
    options: Optional[Dict[str, Any]] = None,
) -> None:
    """Write data using DuckDB.
    
    Args:
        df: DataFrame to write
        connection: Connection object
        format: Output format (csv, parquet)
        table: Table name
        path: File path
        mode: Write mode (overwrite/append)
        options: Format-specific options
    """
    # TODO: Implement
    pass

## 🏗️ Exercise 4: Implement execute_sql()

This is the EASIEST method - DuckDB is SQL-native!

**Pattern:**
```python
# Register all DataFrames from context
for name in context.list_names():
    df = context.get(name)
    conn.register(name, df)

# Execute SQL
result = conn.execute(sql).df()
```

**Tasks:**
1. Register all DataFrames from context
2. Execute the SQL query
3. Return result as Pandas DataFrame

In [ ]:
# Add this method to DuckDBEngine

def execute_sql(self, sql: str, context) -> pd.DataFrame:
    """Execute SQL query using DuckDB.
    
    Args:
        sql: SQL query string
        context: Execution context with registered DataFrames
        
    Returns:
        Result DataFrame
    """
    # TODO: Implement
    pass

## 🏗️ Exercise 5: Implement Introspection Methods

These are simple wrappers around Pandas methods.

**Tasks:**
1. `get_schema()` - Return column names as list
2. `get_shape()` - Return (rows, columns) tuple
3. `count_rows()` - Return row count
4. `count_nulls()` - Return dict of column -> null count
5. `validate_schema()` - Return list of validation errors

In [ ]:
# Add these methods to DuckDBEngine

def get_schema(self, df: pd.DataFrame) -> List[str]:
    """Get DataFrame column names."""
    # TODO: Implement
    pass

def get_shape(self, df: pd.DataFrame) -> tuple:
    """Get DataFrame shape."""
    # TODO: Implement
    pass

def count_rows(self, df: pd.DataFrame) -> int:
    """Count rows in DataFrame."""
    # TODO: Implement
    pass

def count_nulls(self, df: pd.DataFrame, columns: List[str]) -> Dict[str, int]:
    """Count nulls in specified columns."""
    # TODO: Implement (can copy from PandasEngine)
    pass

def validate_schema(self, df: pd.DataFrame, schema_rules: Dict[str, Any]) -> List[str]:
    """Validate DataFrame schema."""
    # TODO: Implement (can copy from PandasEngine)
    pass

## 🏗️ Exercise 6: Implement execute_operation()

For now, just implement pivot (like PandasEngine).

**DuckDB Pivot Pattern:**
```python
# Register DataFrame
conn.register('df', df)

# Execute pivot using DuckDB SQL
sql = f"""
    PIVOT df
    ON {pivot_column}
    USING FIRST({value_column})
    GROUP BY {','.join(group_by)}
"""
result = conn.execute(sql).df()
```

**Tasks:**
1. Extract pivot parameters (group_by, pivot_column, value_column, agg_func)
2. Build DuckDB PIVOT SQL
3. Execute and return result

**Bonus:** Use DuckDB's native PIVOT syntax instead of Pandas!

In [ ]:
# Add this method to DuckDBEngine

def execute_operation(self, operation: str, params: Dict[str, Any], df: pd.DataFrame) -> pd.DataFrame:
    """Execute built-in operation.
    
    Args:
        operation: Operation name
        params: Operation parameters
        df: Input DataFrame
        
    Returns:
        Result DataFrame
    """
    if operation == "pivot":
        # TODO: Implement using DuckDB PIVOT
        pass
    else:
        raise ValueError(f"Unsupported operation: {operation}")

## 🧪 Exercise 7: Test Your Engine

Create comprehensive tests for DuckDBEngine.

**Test Cases:**
1. Read CSV file
2. Write Parquet file
3. Execute SQL query with JOIN
4. Execute SQL with aggregation
5. Pivot operation
6. Schema validation

In [ ]:
# Create test data
import tempfile
from pathlib import Path

# Create temp directory
temp_dir = tempfile.mkdtemp()
print(f"Test directory: {temp_dir}")

# Create sample CSV
test_csv = Path(temp_dir) / "test.csv"
sample_df = pd.DataFrame({
    'id': [1, 2, 3, 4, 5],
    'category': ['A', 'B', 'A', 'B', 'A'],
    'value': [100, 200, 150, 250, 120]
})
sample_df.to_csv(test_csv, index=False)

print(f"Sample data created: {test_csv}")
print(sample_df)

In [ ]:
# TODO: Create mock connection
class MockConnection:
    def __init__(self, base_path):
        self.base_path = Path(base_path)
    
    def get_path(self, path):
        return str(self.base_path / path)

conn = MockConnection(temp_dir)

In [ ]:
# TODO: Test 1 - Read CSV
engine = DuckDBEngine()
df = engine.read(conn, format='csv', path='test.csv')
print("Read test:")
print(df)
assert len(df) == 5
print("✅ Read CSV passed")

In [ ]:
# TODO: Test 2 - Write Parquet
engine.write(df, conn, format='parquet', path='output.parquet')
written_df = engine.read(conn, format='parquet', path='output.parquet')
print("Write test:")
print(written_df)
assert len(written_df) == 5
print("✅ Write Parquet passed")

In [ ]:
# TODO: Test 3 - SQL Aggregation
# Create context (simple dict for now)
from odibi.context import PandasContext

ctx = PandasContext()
ctx.register('sales', df)

result = engine.execute_sql(
    "SELECT category, SUM(value) as total FROM sales GROUP BY category",
    ctx
)
print("SQL aggregation test:")
print(result)
assert len(result) == 2  # Two categories
print("✅ SQL aggregation passed")

In [ ]:
# TODO: Test 4 - Introspection methods
schema = engine.get_schema(df)
print(f"Schema: {schema}")
assert schema == ['id', 'category', 'value']

shape = engine.get_shape(df)
print(f"Shape: {shape}")
assert shape == (5, 3)

rows = engine.count_rows(df)
print(f"Rows: {rows}")
assert rows == 5

print("✅ Introspection methods passed")

## 🏆 Exercise 8: Performance Comparison

Compare DuckDBEngine performance vs PandasEngine.

**Test:** Aggregation on larger dataset

In [ ]:
import time
import numpy as np

# Create larger dataset
large_df = pd.DataFrame({
    'id': range(1000000),
    'category': np.random.choice(['A', 'B', 'C', 'D'], 1000000),
    'value': np.random.randint(1, 1000, 1000000)
})

print(f"Dataset size: {large_df.shape}")

In [ ]:
# Test DuckDB
duck_engine = DuckDBEngine()
duck_ctx = PandasContext()
duck_ctx.register('data', large_df)

start = time.time()
duck_result = duck_engine.execute_sql(
    "SELECT category, AVG(value) as avg_val, COUNT(*) as count FROM data GROUP BY category",
    duck_ctx
)
duck_time = time.time() - start

print(f"DuckDB: {duck_time:.4f} seconds")
print(duck_result)

In [ ]:
# Test Pandas (for comparison)
from odibi.engine.pandas_engine import PandasEngine

pandas_engine = PandasEngine()
pandas_ctx = PandasContext()
pandas_ctx.register('data', large_df)

start = time.time()
pandas_result = pandas_engine.execute_sql(
    "SELECT category, AVG(value) as avg_val, COUNT(*) as count FROM data GROUP BY category",
    pandas_ctx
)
pandas_time = time.time() - start

print(f"Pandas (via DuckDB): {pandas_time:.4f} seconds")
print(pandas_result)

print(f"\n📊 Performance: DuckDB vs Pandas")
print(f"DuckDB: {duck_time:.4f}s")
print(f"Pandas: {pandas_time:.4f}s")
print(f"Speedup: {pandas_time / duck_time:.2f}x")

## 🎯 Bonus Challenges

### Challenge 1: Delta Lake Support
Add Delta Lake reading to DuckDBEngine using DuckDB's `delta_scan()` extension:
```python
conn.execute("INSTALL delta")
conn.execute("LOAD delta")
df = conn.execute("SELECT * FROM delta_scan('path/to/delta')").df()
```

### Challenge 2: Remote File Support
Add S3/Azure support using DuckDB's extensions:
```python
conn.execute("INSTALL azure")
conn.execute("LOAD azure")
conn.execute(f"SET azure_storage_connection_string = '{conn_str}'")
```

### Challenge 3: Query Optimization
Use DuckDB's `EXPLAIN` to analyze query performance:
```python
plan = conn.execute(f"EXPLAIN {sql}").df()
```

### Challenge 4: Streaming Reads
Implement chunked reading for very large files:
```python
# Read in chunks
result = conn.execute(f"SELECT * FROM read_csv_auto('{path}') LIMIT 10000 OFFSET {offset}")
```

## ✅ Completion Checklist

- [ ] DuckDBEngine class created
- [ ] All 9 abstract methods implemented
- [ ] read() supports CSV, Parquet, JSON
- [ ] write() supports CSV, Parquet
- [ ] execute_sql() works with context
- [ ] Introspection methods work
- [ ] Tests pass
- [ ] Performance comparison complete
- [ ] (Bonus) Delta Lake support
- [ ] (Bonus) Remote file support

Once complete, check `solutions.ipynb` for reference implementation!